<h1> Data used</h1> 
Simulated data, which mimics the Starbucks actual data, has been used to solve this capstone project which is in three files given by Udacity.


<h1> Features used to solve the problem:</h1>
      
      1) Attributes related to Customers
           a) Gender
           b) age
           c) income
           d) Year in which he was acquired
           e) month in which he was acquired
      2)  Attributes related to offer
           a) channel: web,social media, mobile,
           b) offer_type
           c) duration
           d) difficulty
           e) reward
           f) Day(weekday or weekend) at which offer is sent 
           g) Month at which offer is sent
      3) Attributes related to historical transaction
           a) total reward per total amount
	       b) Total amount spent on website per month
	       c) Total reward got from website per month
	       d) Total viewed offer per Total sent offer
	       e) Total completed offer per Total sent offer
	       f) Total transaction made per month
          


In [1]:
import pandas as pd
import numpy as np
import math
import json
import datetime as dt
% matplotlib inline

In [2]:
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True) 

In [3]:
#Let's clean the transcript file
transcript['offer_id'] = transcript['value'].apply(lambda x: x['offer id'] if 'offer id' in x.keys() else x['offer_id'] if 'offer_id' in x.keys()  else None)
transcript['amount'] =  transcript['value'].apply(lambda x: x['amount'] if 'amount' in x.keys() else None)
transcript['reward'] = transcript['value'].apply(lambda x: x['reward'] if 'reward' in x.keys() else None)
transcript = transcript[['event','time','person','offer_id','amount','reward']].copy(deep =True)

In [4]:
def f_to_check_number_historical_count_data_related_to_person(x):
    
    person = x['person']
    
    time = x['time']
    
    offer = x['offer_id']
    
    duration = x['duration']*24
    
    time_ = time + duration # time by which offer is valid
    
    df = (transcript.query("person == @person").query("time < @time")).copy(deep=True)
    
    amount = df.amount.sum()
    
    reward = df.reward.sum()
        
    behaviour = dict(df.event.value_counts())
    
    for i in ['transaction', 'offer received', 'offer viewed', 'offer completed']:
        
        if i not in behaviour.keys():
            
            behaviour[i] = None
    
    df2 = list((transcript.query("person == @person").query("@time<time<=@time_ ")).query("offer_id ==@offer").event.unique())
    
    if "offer completed" in df2:
        
        offer_completed = 1
    
    else:
        offer_completed = 0
    
    return pd.Series(  
                      {"Total_amount":amount,
                      'Total_reward':reward,
                      "Total_view":behaviour['offer viewed'],
                      "total_transaction_made":behaviour['transaction'],
                      "Total_offer_received":behaviour['offer received'],
                      "offer completed":behaviour["offer completed"],
                      "is_offer_completed": offer_completed
                       }
        
                     )

In [5]:
received_event = ((transcript[['person','time','offer_id','event']].merge(portfolio.rename(columns = {"id":"offer_id"}),on = "offer_id",how = "left")).query("event == 'offer received' ").query("offer_type != 'informational'").copy(deep =True)[['event', 'time', 'person', 'offer_id','channels','difficulty','duration','offer_type','reward']])

In [7]:
df = pd.concat([received_event,received_event.apply(lambda x: f_to_check_number_historical_count_data_related_to_person(x),axis = 1).fillna(0)],axis = 1) 

In [8]:
for i in ['event','offer_id']:
    del df[i]

In [9]:
df.rename(columns= {"person":"person_id"},inplace = True)

In [10]:
profile.rename(columns= {"id":"person_id"},inplace = True)

In [11]:
df = df.merge(profile,how = "left",on = "person_id")

In [12]:
del df['person_id']

In [13]:
df["Year"] = df["became_member_on"].apply(lambda x: str(dt.datetime.strptime(str(x),"%Y%m%d").year) )

In [14]:
df["month"] = df["became_member_on"].apply(lambda x: str(dt.datetime.strptime(str(x),"%Y%m%d").month) )

In [15]:
df['became_member_on'] = df["became_member_on"].apply(lambda x: dt.datetime.strptime(str(x),"%Y%m%d") )

In [16]:
df['new_time'] = df.apply(lambda x : x['became_member_on'] + dt.timedelta(hours = x['time']),axis = 1)

In [17]:
df["year_in_which_offer_is_made"]= df['new_time'].apply(lambda x : x.year)

In [18]:
df["week_day_in_which_offer_is_made"]= df['new_time'].apply(lambda x : x.weekday())

In [20]:
df["month_in_which_offer_is_made"]= df['new_time'].apply(lambda x : x.month)

In [21]:
df['web'] = df["channels"].apply(lambda x : 1 if "web" in x else 0)

In [22]:
df['email'] = df["channels"].apply(lambda x : 1 if "email" in x else 0)

In [23]:
df['mobile'] = df["channels"].apply(lambda x : 1 if "mobile" in x else 0)

In [24]:
df['social'] = df["channels"].apply(lambda x : 1 if "social" in x else 0)

In [25]:
del df['channels']

In [26]:
df['offer_type'] = df['offer_type'].apply(lambda x: 1 if x == "discount" else 0 )

In [28]:
df.to_csv("data_for_EDA.csv")